Read in PSID data and clean it up.

In [1]:
import pandas as pd
import numpy as np

# read in data - keeping only variabes of interest
psid = pd.read_stata('PS3_data.dta', columns=['id68', 'year', 'hannhrs', 'hlabinc', 'hsex', 'hyrsed', 'age', 'hrace'])

# create wages and ln(wages)
# note need to be careful with wages = 0
psid['wage'] = psid['hlabinc']/psid['hannhrs']
psid['ln_wage'] = np.log(psid['wage'])

# sample selection
psid.drop(psid[psid.hsex != 1].index, inplace=True)
psid.drop(psid[psid.age > 60].index, inplace=True)
psid.drop(psid[psid.age < 25].index, inplace=True)
psid.drop(psid[psid.wage < 7].index, inplace=True)
psid.drop(psid[psid.wage == np.inf].index, inplace=True)

# create dummy variables for race
psid['black'] = (psid['hrace'] == 2).astype(int)
psid['hispanic'] = (psid['hrace'] == 5).astype(int)
psid['other'] = ((psid['hrace'] == 3) | (psid['hrace'] == 4) | (psid['hrace'] == 6) | (psid['hrace'] == 7)).astype(int)

# drop obs if missing values for any variabls in regression model
psid.dropna(axis=0, subset=['ln_wage', 'hyrsed', 'age', 'black', 'other'], inplace=True)

# add a constant
psid['const'] = 1

In [2]:
# Look at data
psid.describe()

,id68,year,hannhrs,hlabinc,hsex,hyrsed,age,hrace,wage,ln_wage,black,hispanic,other,const
count,57097.000000,57097.000000,57097.000000,5.709700e+04,57097.0,57097.000000,57097.000000,57062.000000,57097.000000,57097.000000,57097.000000,57097.0,57097.000000,57097.0
mean,1507.174860,1986.584129,2228.480713,5.282710e+04,1.0,13.529993,39.242939,1.101416,24.320503,3.010798,0.056343,0.0,0.022506,1.0
std,828.407534,8.716500,620.018860,5.235579e+04,0.0,2.449510,9.579581,0.369015,25.204367,0.544119,0.230584,0.0,0.148322,0.0
min,1.000000,1971.000000,2.000000,1.666980e+01,1.0,1.000000,25.000000,1.000000,7.000252,1.945946,0.000000,0.0,0.000000,1.0
25%,782.000000,1979.000000,1952.000000,3.037345e+04,1.0,12.000000,31.000000,1.000000,13.950494,2.635515,0.000000,0.0,0.000000,1.0
50%,1542.000000,1987.000000,2160.000000,4.381145e+04,1.0,13.000000,38.000000,1.000000,19.914677,2.991457,0.000000,0.0,0.000000,1.0
75%,2225.000000,1994.000000,2519.000000,6.138394e+04,1.0,16.000000,47.000000,1.000000,27.793240,3.324793,0.000000,0.0,0.000000,1.0
max,2930.000000,2002.000000,5840.000000,3.771521e+06,1.0,17.000000,60.000000,3.000000,1717.330322,7.448526,1.000000,0.0,1.000000,1.0


In [4]:
## Note - no Hispanics, so exclude that dummy from regression

# create dataframes for the four years of interest
psid1971 = psid[psid['year']==1971].copy()
psid1980 = psid[psid['year']==1980].copy()
psid1990 = psid[psid['year']==1990].copy()
psid2000 = psid[psid['year']==2000].copy()

# OLS estimates
import statsmodels.api as sm
reg1 = sm.OLS(endog=psid1971['ln_wage'], exog=psid1971[['const', 'hyrsed', 'age', 'black', 'other']], missing='drop')
results = reg1.fit()
print('1971 Results: ', results.summary())



1971 Results:                              OLS Regression Results                            
Dep. Variable:                ln_wage   R-squared:                       0.244
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     110.7
Date:                Wed, 13 Sep 2017   Prob (F-statistic):           7.42e-82
Time:                        17:57:17   Log-Likelihood:                -728.06
No. Observations:                1380   AIC:                             1466.
Df Residuals:                    1375   BIC:                             1492.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5510      0.073     

In [65]:
reg2 = sm.OLS(endog=psid1980['ln_wage'], exog=psid1980[['const', 'hyrsed', 'age', 'black', 'other']],
              missing='drop')
results = reg2.fit()
print('1980 Results: ', results.summary())

1980 Results:                              OLS Regression Results                            
Dep. Variable:                ln_wage   R-squared:                       0.169
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     94.08
Date:                Wed, 13 Sep 2017   Prob (F-statistic):           6.40e-73
Time:                        15:21:46   Log-Likelihood:                -1148.4
No. Observations:                1856   AIC:                             2307.
Df Residuals:                    1851   BIC:                             2334.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.6131      0.075     

In [66]:
reg3 = sm.OLS(endog=psid1990['ln_wage'], exog=psid1990[['const', 'hyrsed', 'age', 'black', 'other']],
              missing='drop')
results = reg3.fit()
print('1990 Results: ', results.summary())

1990 Results:                              OLS Regression Results                            
Dep. Variable:                ln_wage   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     139.3
Date:                Wed, 13 Sep 2017   Prob (F-statistic):          3.67e-105
Time:                        15:21:49   Log-Likelihood:                -1393.9
No. Observations:                2013   AIC:                             2798.
Df Residuals:                    2008   BIC:                             2826.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1186      0.084     

In [67]:
reg4 = sm.OLS(endog=psid2000['ln_wage'], exog=psid2000[['const', 'hyrsed', 'age', 'black', 'other']],
              missing='drop')
results = reg4.fit()
print('2000 Results: ', results.summary())

2000 Results:                              OLS Regression Results                            
Dep. Variable:                ln_wage   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     168.6
Date:                Wed, 13 Sep 2017   Prob (F-statistic):          1.75e-128
Time:                        15:21:51   Log-Likelihood:                -2081.4
No. Observations:                2595   AIC:                             4173.
Df Residuals:                    2590   BIC:                             4202.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1611      0.080     

In [27]:
# write likelihood function
def ll_ols(params, data):
    alpha, beta1, beta2, beta3, beta4, sigma = params
    
    n = data['const'].count()
    loglike = (-(n / 2) * np.log(2 * np.pi) - n * np.log(sigma) -
          (1 / (2 * sigma ** 2)) * ((data['ln_wage'] - (alpha * data['const'] +
                                    beta1 * data['hyrsed'] + beta2 * data['age'] +
                                    beta3 * data['black'] + beta4 * data['other'])) ** 2).sum())
    
    return -loglike      

In [32]:
import scipy.optimize as opt

# set initial guesses
# Theta0 = np.ones((6))
#Theta0 = (1.55, 0.069, 0.014, -0.164, 0.030, 1.0)
Theta0 = (1.0, 0.0, 0.0, 0.0, 0.0, 1.0)

# set bounds on parameters
bnds = ((None, None), (None, None), (None, None),
        (None, None), (None, None), (1e-12, None))

# Minimize log-likelihood function for model of each year
ll_results_1971 = opt.minimize(ll_ols, Theta0, args=(psid1971,),
                           method="SLSQP", bounds=bnds, tol=1e-15)
ll_results_1980 = opt.minimize(ll_ols, Theta0, args=(psid1980,),
                           method="SLSQP", bounds=bnds, tol=1e-15)
ll_results_1990 = opt.minimize(ll_ols, Theta0, args=(psid1990,),
                           method="SLSQP", bounds=bnds, tol=1e-15)
ll_results_2000 = opt.minimize(ll_ols, Theta0, args=(psid2000,),
                           method="SLSQP", bounds=bnds, tol=1e-15)

In [35]:
# Put all results together in a dictionary
results_dict = {'1971': ll_results_1971['x'], '1980': ll_results_1980['x'],
                '1990': ll_results_1990['x'], '2000': ll_results_2000['x']}
for y in ['1971', '1980', '1990', '2000']:
    print('Return to education in ', y, " = ", results_dict[y][1])

Return to education in  1971  =  0.0668310500949
Return to education in  1980  =  0.0683662333218
Return to education in  1990  =  0.0875825705465
Return to education in  2000  =  0.117637843589


In [123]:
# OLS via matrix algebra
X = np.array(psid1971[['const', 'hyrsed', 'age', 'black', 'other']])
Y = np.array(psid1971[['ln_wage']])
ols_mat = np.dot(np.linalg.inv(np.dot(np.transpose(X),X)), np.dot(np.transpose(X),Y))
print(ols_mat)

[[ 1.55096436]
 [ 0.06687862]
 [ 0.0143913 ]
 [-0.16388845]
 [ 0.03068709]]
